均线 一 上证信息

我是格子衫小C（微信公众号：格子衫小C），今天开始，我们一起来研究均线指标（Indicator）。

均线是最常见的一种指标，一般交易软件里都能看到。

均线，顾名思义，就是用各种加权算法算出历史平均价格。

因为均线有明显的滞后性（譬如用前5日价格算出来的平均数，高点或低点都一定是滞后的），所以一般叫 移动平均线，也就是 Moving Averages ，简称 MA。

MA里，平均数的算法有很多种，因此MA也有很多种，其中，最简单的就是算术平均数算出来的，一般叫 MA 或 SMA。

In [ ]:
import pandas as pd
from datetime import datetime
import trdb2py

isStaticImg = False
width = 960
height = 768

pd.options.display.max_columns = None
pd.options.display.max_rows = None

trdb2cfg = trdb2py.loadConfig('./trdb2.yaml')

In [ ]:
# 具体基金
# asset = 'jrj.510310'
asset = 'jqdata.000039_XSHG|1d'

# 起始时间，0表示从最开始算起
# tsStart = 0
tsStart = int(trdb2py.str2timestamp('2013-05-01', '%Y-%m-%d'))

# 结束时间，-1表示到现在为止
# tsEnd = -1
tsEnd = int(trdb2py.str2timestamp('2020-09-30', '%Y-%m-%d'))

# 初始资金池
paramsinit = trdb2py.trading2_pb2.InitParams(
    money=10000,
)

# 买入参数，用全部的钱来买入（也就是复利）
paramsbuy = trdb2py.trading2_pb2.BuyParams(
    perHandMoney=1,
)

# 卖出参数，全部卖出
paramssell = trdb2py.trading2_pb2.SellParams(
    perVolume=1,
)

In [ ]:
# baseline    
s0 = trdb2py.trading2_pb2.Strategy(
    name="normal",
    asset=trdb2py.str2asset(asset),         
)
        
buy0 = trdb2py.trading2_pb2.CtrlCondition(
    name='buyandhold',
)

paramsbuy = trdb2py.trading2_pb2.BuyParams(
    perHandMoney=1,
)

paramsinit = trdb2py.trading2_pb2.InitParams(
    money=10000,
)

s0.buy.extend([buy0])
s0.paramsBuy.CopyFrom(paramsbuy)
s0.paramsInit.CopyFrom(paramsinit)        
p0 = trdb2py.trading2_pb2.SimTradingParams(
    assets=[trdb2py.str2asset(asset)],
    startTs=tsStart,
    endTs=tsEnd,
    strategies=[s0],
    title='baseline',
)  

pnlBaseline = trdb2py.simTrading(trdb2cfg, p0)
trdb2py.showPNL(pnlBaseline, toImg=isStaticImg, width=width, height=height)

我们还是拿沪深300指数，2013年5月1日到2020年9月30日的日线数据来做例子。

In [ ]:
ret = trdb2py.getAssetCandles2(trdb2cfg, asset, tsStart, tsEnd, indicators=['sma.2', 'sma.5', 'sma.30', 'sma.60', 'sma.120', 'sma.240'])
# print(ret) 
# ret
trdb2py.showAssetCandles2('沪深300指数', ret, indicators=['sma.2', 'sma.5', 'sma.30', 'sma.60', 'sma.120', 'sma.240'], toImg=isStaticImg, width=width, height=height)

上面可以看到30日、60日、120日、240日均线，可以看到，随着容纳的数据变多，均线是越来越平滑的。

1. 越大的均线越平滑
2. 越大的均线高点越低，低点越高
3. 越长的均线滞后性越明显

将均线和价格曲线对比来看，我们能发现

1. 上涨时，价格曲线在均线上方（均线的滞后性）
2. 下跌时，价格曲线在下方

看起来，只要找到一条合适的均线，在价格向上穿过均线时买入，价格向下穿过均线时卖出，但其实没这么简单。

越长的均线相交点越少，但滞后越严重，而越短的均线，很容易有误穿的情况。

而且，理论上，这个均线参数不会适用所有情况，哪怕是一种资产的均线。

接下来，测一下均线

In [ ]:
lstparams = []

# 具体基金
# asset = 'jrj.510310'
asset = 'jqdata.000039_XSHG|1d'

# 起始时间，0表示从最开始算起
# tsStart = 0
tsStart = int(trdb2py.str2timestamp('2013-05-01', '%Y-%m-%d'))

# 结束时间，-1表示到现在为止
# tsEnd = -1
tsEnd = int(trdb2py.str2timestamp('2020-09-30', '%Y-%m-%d'))

for ema in range(2, 241):    
    buy0 = trdb2py.trading2_pb2.CtrlCondition(
        name='indicatorsp',
        operators=['upcross'],
        strVals=['ta-sma.{}'.format(ema)],
    )
    
    buy1 = trdb2py.trading2_pb2.CtrlCondition(
        name='waittostart',
        vals=[ema],
    )    
            
    sell0 = trdb2py.trading2_pb2.CtrlCondition(
        name='indicatorsp',
        operators=['downcross'],
        strVals=['ta-sma.{}'.format(ema)],
    )
    
    s0 = trdb2py.trading2_pb2.Strategy(
        name="normal",
        asset=trdb2py.str2asset(asset),
    )

    s0.buy.extend([buy0, buy1])
    s0.sell.extend([sell0])
    s0.paramsBuy.CopyFrom(paramsbuy)
    s0.paramsSell.CopyFrom(paramssell) 
    s0.paramsInit.CopyFrom(paramsinit)        
    lstparams.append(trdb2py.trading2_pb2.SimTradingParams(
        assets=[trdb2py.str2asset(asset)],
        startTs=tsStart,
        endTs=tsEnd,
        strategies=[s0],
        title='sma.{}'.format(ema),
    ))
 
lstpnlmix = trdb2py.simTradings(trdb2cfg, lstparams, ignoreTotalReturn=2.3)

trdb2py.showPNLs(lstpnlmix + [pnlBaseline], toImg=isStaticImg, width=width, height=height)

In [ ]:
dfpnl = trdb2py.buildPNLReport(lstpnlmix + [pnlBaseline])

# dfpnl1 = dfpnl[dfpnl['totalReturns'] >= 2]

dfpnl[['title', 'maxDrawdown', 'maxDrawdownStart', 'maxDrawdownEnd', 'totalReturns', 'sharpe', 'annualizedReturns', 'annualizedVolatility', 'variance']].sort_values(by='totalReturns', ascending=False)

可以看到，简单的均线策略，最好的情况也就是将上涨的趋势把握住，跳过下跌大的区间，当然，基本上上涨和下跌都会有损失。

这里粒度非常重要，看估计很难看出来，只能试。

In [ ]:
# lstems = [2, 42, 60]
# 起始时间，0表示从最开始算起
tsStart = 0
# tsStart = int(trdb2py.str2timestamp('2013-05-01', '%Y-%m-%d'))

# 结束时间，-1表示到现在为止
tsEnd = -1
# tsEnd = int(trdb2py.str2timestamp('2020-09-30', '%Y-%m-%d'))

# baseline    
s0 = trdb2py.trading2_pb2.Strategy(
    name="normal",
    asset=trdb2py.str2asset(asset),         
)
        
buy0 = trdb2py.trading2_pb2.CtrlCondition(
    name='buyandhold',
)

paramsbuy = trdb2py.trading2_pb2.BuyParams(
    perHandMoney=1,
)

paramsinit = trdb2py.trading2_pb2.InitParams(
    money=10000,
)

s0.buy.extend([buy0])
s0.paramsBuy.CopyFrom(paramsbuy)
s0.paramsInit.CopyFrom(paramsinit)        
p0 = trdb2py.trading2_pb2.SimTradingParams(
    assets=[trdb2py.str2asset(asset)],
    startTs=tsStart,
    endTs=tsEnd,
    strategies=[s0],
    title='baseline',
)  

pnlBaselineF = trdb2py.simTrading(trdb2cfg, p0)
trdb2py.showPNL(pnlBaselineF, toImg=isStaticImg, width=width, height=height)

In [ ]:
lstparams = []

lstems = [27, 41, 47]
# 起始时间，0表示从最开始算起
tsStart = 0
# tsStart = int(trdb2py.str2timestamp('2013-05-01', '%Y-%m-%d'))

# 结束时间，-1表示到现在为止
tsEnd = -1
# tsEnd = int(trdb2py.str2timestamp('2020-09-30', '%Y-%m-%d'))

for ema in lstems:    
    buy0 = trdb2py.trading2_pb2.CtrlCondition(
        name='indicatorsp',
        operators=['upcross'],
        strVals=['ta-sma.{}'.format(ema)],
    )
    
    buy1 = trdb2py.trading2_pb2.CtrlCondition(
        name='waittostart',
        vals=[ema],
    )    
            
    sell0 = trdb2py.trading2_pb2.CtrlCondition(
        name='indicatorsp',
        operators=['downcross'],
        strVals=['ta-sma.{}'.format(ema)],
    )
    
    s0 = trdb2py.trading2_pb2.Strategy(
        name="normal",
        asset=trdb2py.str2asset(asset),
    )

    s0.buy.extend([buy0, buy1])
    s0.sell.extend([sell0])
    s0.paramsBuy.CopyFrom(paramsbuy)
    s0.paramsSell.CopyFrom(paramssell) 
    s0.paramsInit.CopyFrom(paramsinit)        
    lstparams.append(trdb2py.trading2_pb2.SimTradingParams(
        assets=[trdb2py.str2asset(asset)],
        startTs=tsStart,
        endTs=tsEnd,
        strategies=[s0],
        title='sma.{}'.format(ema),
    ))
 
lstpnlmix = trdb2py.simTradings(trdb2cfg, lstparams, ignoreTotalReturn=2.3)

trdb2py.showPNLs(lstpnlmix + [pnlBaselineF], toImg=isStaticImg, width=width, height=height)

In [ ]:
dfpnl = trdb2py.buildPNLReport(lstpnlmix + [pnlBaselineF])

# dfpnl1 = dfpnl[dfpnl['totalReturns'] >= 2]

dfpnl[['title', 'maxDrawdown', 'maxDrawdownStart', 'maxDrawdownEnd', 'totalReturns', 'sharpe', 'annualizedReturns', 'annualizedVolatility', 'variance']].sort_values(by='totalReturns', ascending=False)